## 第八章 特征提取

本章的特征提取指的是特征变换。

特征变换和前面的特征选择的目的都是将特征降维，但是特征选择是直接删掉一些特征，而特征提取是通过数学方法把高维特征映射为低维。

### 一、基于类别可分性判据的特征提取

这一节介绍的方法是通过最大化类内类间可分性判据$J$来对特征进行线性变换降维，也就是找到一个$\mathbf{W}$，将样本$\mathbf{x}$投影为$\mathbf{y = \mathbf{W}^\text{T}\mathbf{x}}$，用投影后的$\mathbf{y}$算出来的可分性判据$J$最大。

书上的推导过程需要先学会标量对矩阵求导，我在[补充的数学知识.ipynb](./补充的数学知识.ipynb)中有介绍。

经过一系列推导，最终的方法是：

1. 求出矩阵$\mathbf{S}_{\text{w}}^{-1}\mathbf{S}_{\text{b}}$。

关于$\mathbf{S}_{\text{w}}$和$\mathbf{S}_{\text{b}}$，书上在第四章和第七章中的表述不一致，我认为应该这里的公式应该采取第七章的说法，即

$$\begin{align}
\mathbf{S}_{\text{b}} &= \sum_{i=1}^cP_i(\mathbf{m}_i - \mathbf{m})(\mathbf{m}_i - \mathbf{m})^\text{T} \\
\mathbf{S}_{\text{w}} &= \sum_{i=1}^cP_i\frac{1}{n_i}\sum_{k=1}^{n_i}(\mathbf{x}_k^{(i)}-\mathbf{m}_i)(\mathbf{x}_k^{(i)}-\mathbf{m}_i)^\text{T}
\end{align}$$

其中$P_i$采用样本中类别的比例来进行估算，其实如果这样算的话公式里的$P_i\frac{1}{n_i}$就变成了$\frac{1}{n}$。

2. 求出$\mathbf{S}_{\text{w}}^{-1}\mathbf{S}_{\text{b}}$的特征值和特征向量。
3. 把特征值从大到小排序，选取前面最大的$d$个特征值的特征向量作为$\mathbf{W}$。$d$是人为划定的，你想把样本降维成几维，$d$就是多大。

计算特征值可以用`np.linalg.eig`方法来实现。`np.linalg.eig`返回值包含两部分，特征值和每个特征值对应的列向量（注意这里是列向量）。

接下来是代码实现，还是注意和前几章同样的问题，numpy中向量是行向量，代码中实现的矩阵是公式中矩阵的转置，除了`np.linalg.eig`返回的特征向量矩阵。

In [14]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
def get_mean(_X: np.ndarray) -> np.ndarray:
    """
    计算m_i（样本均值）
    :param _X: 样本矩阵，每一行是一个样本
    :return: 均值（一维向量）
    """
    return np.mean(_X, 0)


def get_within_class_scatter_matrix(_X: np.ndarray) -> np.ndarray:
    """
    计算某一类的S_w_i（类内离散度矩阵）
    :param _X: 样本矩阵，每一行是一个样本
    :return: 类内离散度矩阵（D * D）
    """
    ret = np.zeros((_X.shape[1], _X.shape[1]))
    m = get_mean(_X)
    for row in _X:
        ret += (row - m).reshape(m.shape[0], 1) @ (row - m).reshape(m.shape[0], 1).T
    return ret


def get_pooled_within_class_scatter_matrix(_X1: np.ndarray, _X2: np.ndarray) -> np.ndarray:
    """
    计算最终的S_w（总类内离散度矩阵）
    :param _X1: 第一类样本矩阵，每一行是一个样本
    :param _X2: 第二类样本矩阵，每一行是一个样本
    :return: 总类内离散度矩阵（D * D）
    """
    return get_within_class_scatter_matrix(_X1) + get_within_class_scatter_matrix(_X2) / (_X1.shape[0] + _X2.shape[0])

def get_between_class_scatter_matrix(_X1: np.ndarray, _X2: np.ndarray) -> np.ndarray:
    """
    计算S_b（类间离散度矩阵）
    :param _X1: 第一类样本矩阵，每一行是一个样本
    :param _X2: 第二类样本矩阵，每一行是一个样本
    :return: 类间离散度矩阵（D * D）
    """
    n1 = _X1.shape[0]
    n2 = _X2.shape[0]
    d = _X1.shape[1]
    N = n1 + n2
    m_1 = get_mean(_X1)
    m_2 = get_mean(_X2)
    m = get_mean(np.concatenate((_X1, _X2)))
    return n1 / N * ((m_1 - m).reshape(d, 1) @ (m_1 - m).reshape(d, 1).T) + \
        n2 / N * ((m_2 - m).reshape(d, 1) @ (m_2 - m).reshape(d, 1).T)

def get_W(_X1: np.ndarray, _X2: np.ndarray, d: int) -> np.ndarray:
    """
    计算最终的W矩阵
    :param _X1: 第一类样本矩阵，每一行是一个样本
    :param _X2: 第二类样本矩阵，每一行是一个样本
    :param d: 最终要投影的特征维度数量
    :return: W矩阵
    """
    S_w_inv_ = np.linalg.inv(get_pooled_within_class_scatter_matrix(_X1, _X2))
    S_b_ = get_between_class_scatter_matrix(_X1, _X2)

    # np.np.linalg.eig用来计算特征值和特征向量，返回的结果可能是复数
    eigen_values_, eigen_vectors_ = np.linalg.eig(S_w_inv_ @ S_b_)

    # 下面用来对特征值排序，并且记录特征值的序号
    eigen_temp_ = []
    for i in range(eigen_values_.shape[0]):
        # 找到是实数的特征值
        if eigen_values_[i].imag == 0:
            eigen_temp_.append([eigen_values_[i].real, i])

    eigen_temp_.sort(reverse=True)

    # 下面用来找出最大的d个实特征值的特征向量
    ret = []
    for i in range(d):
        ret.append(eigen_vectors_[:,eigen_temp_[i][1]].real)
    return np.array(ret)

In [3]:
# 加载数据，这次的数据用sklearn自带的水仙花的数据
iris = load_iris()
X = iris.data[:100]
y = iris.target[:100]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 寻找两类样本
X1 = []
X2 = []
for i in range(X_train.shape[0]):
    if y_train[i] == 0:
        X1.append(X_train[i])
    else:
        X2.append(X_train[i])
X1 = np.array(X1)
X2 = np.array(X2)

In [5]:
# 计算W, d设置为1，即投影后的新特征为1维
W = get_W(X1, X2, 1)
print("W为：")
print(W)

W为：
[[ 0.1387983  -0.23718357  0.74740629  0.60486595]]


接下来生成降维后的数据，用支持向量机训练测试一下效果。

In [6]:
X_train_new = X_train @ W.T
X_test_new = X_test @ W.T
print("降维之后的样本维度：", X_train_new.shape[1])

svm_classifier = SVC(kernel="rbf")  # 用径向基函数作为核函数
svm_classifier.fit(X_train_new, y_train)

print("降维之后的正确率：", svm_classifier.score(X_test_new, y_test))

降维之后的样本维度： 1
降维之后的正确率： 1.0


测试结果表明，用基于分类可分性判据的特征提取方法，就算我们把样本从四维降成一维，还能保持100%的正确率。这说明这个降维方法是非常有效的。

### 二、主成分分析方法

主成分分析是根据方差大小来对样本进行线性变换，最终实现对样本的降维。每一个主成分都是样本协方差矩阵的一个特征值。

主成分分析的公式推导并不复杂，书上介绍的也很清楚。

主要的问题在于主成分分析没有考虑样本分类信息，只是按照样本方差来处理数据，这样的方法只是把样本降维了，但是不一定对分类有利。

主成分分析实际上是K-L变换的一种特殊情况，下面会实现K-L变换，因此这里我就不写代码实现了。

### 三、K-L变换

原始的K-L变换的根据最小均方误差来进行线性变换，如果样本均值为$0$，那么K-L变换等价于主成分分析。

为了应用于模式识别，需要考虑到样本的分类信息，K-L变换引入了新的方法。

#### 1. 从类均值中提取判别信息

这个方法是假设样本的主要分类信息包含在均值中，通过总类内离散度矩阵$\mathbf{S}_{\text{w}}$作为产生矩阵进行K-L变换。然后在根据可分性判据大小得出新特征的转换矩阵。

具体方法如下：

1. 计算总类内离散度矩阵$\mathbf{S}_{\text{w}}$和类间离散度矩阵$\mathbf{S}_{\text{b}}$。
2. 用$\mathbf{S}_{\text{w}}$进行K-L变换，得到特征值$\lambda_i$和对应的特征向量$\mathbf{u}_i$（一个特征维度对应一个特征值和特征向量）。
3. 通过特征值和特征向量计算样本每个维度的可分性判据大小，公式如下：

$$
J(y_i) = \frac{\mathbf{u}_i^\text{T}\mathbf{S}_{\text{b}}\mathbf{u}_i}{\lambda_i}
$$

4. 找出$J(y_i)$最大的$d$个（d是要降低的目标维数）特征向量$\mathbf{u}_1,\mathbf{u}_2,\cdots, \mathbf{u}_d$，转换矩阵就是$\mathbf{U} = [\mathbf{u}_1,\mathbf{u}_2,\cdots, \mathbf{u}_d]$，降维后的样本就是$\mathbf{y} = \mathbf{U}^\text{T}\mathbf{x}$。

算法实现如下：

In [22]:
# 数据在上面已经生成过了，这里直接用上面生成的数据
S_w = get_pooled_within_class_scatter_matrix(X1, X2)
S_b = get_between_class_scatter_matrix(X1, X2)

# 注意numpy生成的特征向量矩阵中，每一列是一个特征向量
eigen_values, eigen_vectors = np.linalg.eig(S_w)

# np.diag把方阵的对角元素提取出来，返回一个向量
J = np.diag(eigen_vectors.T @ S_b @ eigen_vectors) / eigen_values
print("可分性判据J数值如下：", J)

可分性判据J数值如下： [0.00452935 0.07519813 0.19099989 1.620232  ]


In [23]:
# 接下来找出J最大的d个特征向量，生成转换矩阵U
# 这个例子中我们把原来4维的样本降成1维，所以d=1，那么只需要找出最大的J对应的特征向量即可

maxn = -999999
max_index = -1
for i in range(J.shape[0]):
    if J[i] > maxn:
        maxn = J[i]
        max_index = i

U = eigen_vectors[:, max_index].reshape(J.shape[0])
print("我们要把样本降维成一维，所以转换矩阵U退化成一个向量，U的值为：")
print(U)

我们要把样本降维成一维，所以转换矩阵U退化成一个向量，U的值为：
[-0.42773576  0.47409999 -0.73338996 -0.23326053]


接下来把生成降维后的样本，再用SVM测试一下。

In [24]:
X_train_new = X_train @ U.reshape(U.shape[0], 1)
X_test_new = X_test @ U.reshape(U.shape[0], 1)
print("降维之后的样本维度：", X_train_new.shape[1])

svm_classifier = SVC(kernel="rbf")  # 用径向基函数作为核函数
svm_classifier.fit(X_train_new, y_train)

print("降维之后的正确率：", svm_classifier.score(X_test_new, y_test))

降维之后的样本维度： 1
降维之后的正确率： 1.0


显然，从类均值中提取判别信息的降维效果也很好。

#### 2. 包含在类平均向量中判别信息的最优压缩

这个方法是使特征间互不相关的前提下最优压缩均值向量中包含的分类信息。

具体方法如下：

1. 求出$\mathbf{S}_{\text{w}}$和$\mathbf{S}_{\text{b}}$
2. 对$\mathbf{S}_{\text{w}}$做K-L变换，求出特征变换矩阵$\mathbf{U}$
3. 令$\mathbf{B}=\mathbf{U}\boldsymbol{\Lambda}^{-\frac{1}{2}}$，这个时候$\mathbf{B}^\text{T}\mathbf{S}_{\text{w}}\mathbf{B} = \mathbf{I}$，这一步成为白化变换
4. 求$\mathbf{S}_{\text{b}}' = \mathbf{B}^\text{T}\mathbf{S}_{\text{b}}\mathbf{B}$
5. 对$\mathbf{S}_{\text{b}}'$进行K-L变换，求出特征变换矩阵$\mathbf{V}'$，其中$\mathbf{V}'$中只包含$\mathbf{S}_{\text{b}}'$中非零的实特征值对应的特征向量（$c$类分类问题最多有$c-1$个）
6. 最终总的变换矩阵是$\mathbf{W} = \mathbf{U}\boldsymbol{\Lambda}^{-\frac{1}{2}}\mathbf{V}'$

最终得到的投影方向实际上就是Fisher线性判别的投影方向。

这个方法没有指定到底降维成多少维，我们测试一下实际的降维效果。

In [28]:
# 注意下面这段代码所有的矩阵都是公式中的矩阵，而不像之前一样是公式中矩阵的转置

S_w = get_pooled_within_class_scatter_matrix(X1, X2)
S_b = get_between_class_scatter_matrix(X1, X2)
eigen_values, eigen_vectors = np.linalg.eig(S_w)
U = eigen_vectors
Lambda_of_neg_1_2 = np.diag(eigen_values ** (-1 / 2))
B = U @ np.linalg.inv(Lambda_of_neg_1_2)
S_b_1 = B.T @ S_b @ B
eigen_values, eigen_vectors = np.linalg.eig(S_b_1)
V_1 = []
for i in range(eigen_values.shape[0]):
    if eigen_values[i] != 0:
        V_1.append(eigen_vectors[:, i].reshape(eigen_vectors.shape[0]))
V_1 = np.array(V_1).T
W = U @ Lambda_of_neg_1_2 @ V_1
print("最终计算出的投影方向W为：")
print(W)

最终计算出的投影方向W为：
[[ 0.2296443  -0.68162292  0.14097299]
 [-0.16832518  0.40523553  0.31917842]
 [ 0.70310732  0.24704072 -0.18799676]
 [ 0.26937611  0.56457478 -0.11018427]]


显然最后样本被降维成三维。这个降维可以理解为保留了样本几乎全部信息，虽然最终维度比较高，但是分类信息保留全面。

#### 3. 类中心化特征向量中分类信息的提取

这个方法时考虑从样本的各类协方差中提取信息，具体方式是：先对$\mathbf{S}_{\text{w}}$进行K-L变换，然后在根据新样本的各个维度方差的大小来计算可分性判据$J(x_j)$。最后取$J(x_j)$最大的前d个特征。

这个方法可以和前面介绍的第二种方法结合起来，先用第二种进行均值分类信息最优压缩，压缩后获得$d' \leqslant c-1$个特征，再用第三种方法压缩获得另外$d - d'$个特征。

算法的实现也和前面的大同小异，这里我就不实现了。

### 四、多维尺度法（MDS）

MDS运用了缩放的思想，把高维空间的样本按比例缩放到低维空间，书上用地图的例子很好的解释了这个思想。

#### 1. 古典尺度法

古典尺度法就是已知各样本间的欧式距离，求他们在指定维度下的坐标。

实现方法如下：

1. 定义中心化矩阵$\mathbf{J} = \mathbf{I} - \frac{1}{n}\mathbf{11}^\text{T}$
2. 计算欧氏距离矩阵$\mathbf{D}^{(2)}$
3. 求样本两两内积组成的矩阵$\mathbf{B} = -\frac{1}{2}\mathbf{JDJ}$
4. 求$\mathbf{B}$的特征值$\lambda_1,\lambda_2,\cdots,\lambda_d$和特征向量$\mathbf{u}_1,\mathbf{u}_2,\cdots,\mathbf{u}_d$
5. 假设要将样本降维到$k$维，则将特征值从大到小排序，取前$k$大的特征值对应的特征向量组成矩阵$\mathbf{U}=[\mathbf{u}_1,\mathbf{u}_2,\cdots,\mathbf{u}_k]$
6. 解出样本坐标矩阵$\mathbf{X}=\mathbf{U}\boldsymbol{\Lambda}^{1/2}$
7. 将样本从中心化还原，$\widetilde{\mathbf{x}}_i = \mathbf{x}_i + \overline{\mathbf{x}}$

这个方法其实和主成分分析是相同的，代码实现如下：

In [20]:
n = X.shape[0]
J = np.eye(n) - 1 / n * np.ones(n)
D = X @ X.T
B = -1 / 2 * J @ D @ J
eigen_values, eigen_vectors = np.linalg.eig(B)

# 下面用来对特征值排序，并且记录特征值的序号
eigen_temp = []
for i in range(eigen_values.shape[0]):
    # 找到是实数的特征值
    if eigen_values[i].imag == 0:
        eigen_temp.append([eigen_values[i].real, i])
eigen_temp.sort(reverse=True)

# 下面用来找出最大的d个实特征值的特征向量，假设d=1
U = []
Lambda_of_1_2 = []
for i in range(1):
    U.append(eigen_vectors[:,eigen_temp[i][1]].real.T)
    Lambda_of_1_2.append(np.sqrt(eigen_temp[i][0].real))
U = np.array(U).T
Lambda_of_1_2 = np.array(Lambda_of_1_2)
X_new = U @ Lambda_of_1_2
X_new = X_new + get_mean(X_new)
print(X_new)

[-2.27826921e-08 -3.44144050e-08 -1.75140000e-08  2.28469080e-09
 -1.67971705e-08  5.34840161e-08  3.43430899e-09  2.98690681e-08
 -2.13786560e-08  1.63238719e-08  4.11062135e-08 -1.98371979e-08
 -2.90794537e-08  8.90252228e-09  2.37053957e-09  1.17263691e-08
  2.07078762e-08 -9.75305242e-10  3.20626111e-08  2.71718079e-08
  3.33402252e-08  1.26129911e-08 -3.12520589e-08 -2.70509339e-09
 -4.24973096e-08 -1.35221505e-08  7.88039006e-09 -2.87083186e-09
 -3.48658361e-08 -8.55255398e-09 -9.89458731e-09  2.34282814e-08
  1.08033115e-08 -3.47947733e-08 -5.32301673e-09 -5.81293064e-09
 -2.45663341e-08 -2.86082110e-08 -4.12486016e-08 -8.36352275e-09
 -1.87246325e-08 -1.01729126e-08 -3.10669694e-08 -1.81945259e-08
 -1.08101679e-08 -3.65419330e-08 -1.08827192e-08 -4.30372786e-08
  4.86084062e-09 -2.70163812e-08  4.15228906e-08 -6.41982072e-09
 -1.74987220e-08  2.09008665e-08 -2.50100769e-08 -8.61789227e-09
 -5.92910143e-09 -4.81297694e-09 -6.88161834e-09  4.16202265e-08
  1.64202060e-09  1.42302

这个方法我就不测试了，和前面的都一样。

#### 2. 度量型MSD和非度量型MSD

这两个书上都只是简单的介绍了一下，看一下就行。这类算法的特点是实际实现没有解析解，要使用梯度下降来计算。

### 三、核主成分分析

核主成分分析的思想是通过核函数把样本投影到高维，这样原来非线性的规律也变成了线性的，再通过进行线性的主成分分析。

思想很简单，实现也很简单。鉴于书上的篇幅有限，这里也就不实现了。在sklearn中有写好的KPCA库可以用。

### 四、IspMap方法和LLE方法

这个方法就比较抽象了，主要思想是采取分治，把数据切割成一个一个小块，对每个小块求平均抽象成新的样本，然后再进行映射。

书上只是简单介绍了一下思想，了解一下就行。